In [10]:
import pandas as pd
df = pd.read_csv("../datasets/books_data.csv")
df

,BookID,FormattedBookID,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,1,BK000001,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,9278135,https://www.goodreads.com/book/show/72193.Harr...
1,2,BK000002,"Hungerspelen (The Hunger Games, #1)",Suzanne Collins,24 deltagare. Bara en överlever. Varje år dire...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963186,https://www.goodreads.com/book/show/13616052-h...
2,3,BK000003,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,"Could you survive on your own in the wild, wit...","['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963186,https://www.goodreads.com/book/show/12187803-t...
3,4,BK000004,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,In the ruins of a place once known as North Am...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963186,https://www.goodreads.com/book/show/18626858-t...
4,5,BK000005,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,Sometime in the future after the destruction o...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963186,https://www.goodreads.com/book/show/12091570-t...
...,...,...,...,...,...,...,...,...,...
9030,9031,BK009031,The Alchemy Of The Soul,Vikram Joshi,I feel that I was partly responsible for my fa...,['Spirituality'],4.25,4,https://www.goodreads.com/book/show/13629106-t...
9031,9032,BK009032,A Hummen in Spiral Gorge (Stones of Elemdray #1),T. William Watts,"“No common dreams, no common pools,For these t...","['Childrens', 'Middle Grade', 'Young Readers']",5.00,4,https://www.goodreads.com/book/show/17189921-a...
9032,9033,BK009033,The Travels of Marco Polo,Giulio Einaudi,"Marco Polo, a Medieval Venetian, traveled thro...","['Travel', 'History']",3.00,4,https://www.goodreads.com/book/show/15790154-t...
9033,9034,BK009034,Parenting Your Child Leader: Strategies for He...,Darryl Bodkin,p>Darryl Bodkin has written a wonderful treati...,['Parenting'],4.50,4,https://www.goodreads.com/book/show/17989576-p...


In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import ChatHuggingFace
import faiss
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
load_dotenv()

True

In [45]:
df["tagged_desc"] = df["FormattedBookID"].astype(str) + " " + df["Description"].fillna("")


In [46]:
df

,BookID,FormattedBookID,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL,tagged_desc
0,1,BK000001,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,9278135,https://www.goodreads.com/book/show/72193.Harr...,BK000001 Harry Potter thinks he is an ordinary...
1,2,BK000002,"Hungerspelen (The Hunger Games, #1)",Suzanne Collins,24 deltagare. Bara en överlever. Varje år dire...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963186,https://www.goodreads.com/book/show/13616052-h...,BK000002 24 deltagare. Bara en överlever. Varj...
2,3,BK000003,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,"Could you survive on your own in the wild, wit...","['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963186,https://www.goodreads.com/book/show/12187803-t...,BK000003 Could you survive on your own in the ...
3,4,BK000004,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,In the ruins of a place once known as North Am...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963186,https://www.goodreads.com/book/show/18626858-t...,BK000004 In the ruins of a place once known as...
4,5,BK000005,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,Sometime in the future after the destruction o...,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...",4.33,7963186,https://www.goodreads.com/book/show/12091570-t...,BK000005 Sometime in the future after the dest...
...,...,...,...,...,...,...,...,...,...,...
9030,9031,BK009031,The Alchemy Of The Soul,Vikram Joshi,I feel that I was partly responsible for my fa...,['Spirituality'],4.25,4,https://www.goodreads.com/book/show/13629106-t...,BK009031 I feel that I was partly responsible ...
9031,9032,BK009032,A Hummen in Spiral Gorge (Stones of Elemdray #1),T. William Watts,"“No common dreams, no common pools,For these t...","['Childrens', 'Middle Grade', 'Young Readers']",5.00,4,https://www.goodreads.com/book/show/17189921-a...,"BK009032 “No common dreams, no common pools,Fo..."
9032,9033,BK009033,The Travels of Marco Polo,Giulio Einaudi,"Marco Polo, a Medieval Venetian, traveled thro...","['Travel', 'History']",3.00,4,https://www.goodreads.com/book/show/15790154-t...,"BK009033 Marco Polo, a Medieval Venetian, trav..."
9033,9034,BK009034,Parenting Your Child Leader: Strategies for He...,Darryl Bodkin,p>Darryl Bodkin has written a wonderful treati...,['Parenting'],4.50,4,https://www.goodreads.com/book/show/17989576-p...,BK009034 p>Darryl Bodkin has written a wonderf...


In [47]:
df["tagged_desc"]

0       BK000001 Harry Potter thinks he is an ordinary...
1       BK000002 24 deltagare. Bara en överlever. Varj...
2       BK000003 Could you survive on your own in the ...
3       BK000004 In the ruins of a place once known as...
4       BK000005 Sometime in the future after the dest...
                              ...                        
9030    BK009031 I feel that I was partly responsible ...
9031    BK009032 “No common dreams, no common pools,Fo...
9032    BK009033 Marco Polo, a Medieval Venetian, trav...
9033    BK009034 p>Darryl Bodkin has written a wonderf...
9034    BK009035 One island, dangerous on the outside,...
Name: tagged_desc, Length: 9035, dtype: object

In [48]:
df["tagged_desc"].to_csv("tagged_desc.txt",
                         index = False,
                         sep="\n",
                         header=False)

In [49]:
raw_docs = TextLoader("tagged_desc.txt",encoding="utf-8").load()
text_splitters = CharacterTextSplitter(chunk_size=1500,chunk_overlap=0,separator="\n")
documents = text_splitters.split_documents(raw_docs)

Created a chunk of size 1812, which is longer than the specified 1500
Created a chunk of size 1621, which is longer than the specified 1500
Created a chunk of size 1591, which is longer than the specified 1500
Created a chunk of size 1594, which is longer than the specified 1500
Created a chunk of size 1930, which is longer than the specified 1500
Created a chunk of size 1657, which is longer than the specified 1500
Created a chunk of size 1524, which is longer than the specified 1500
Created a chunk of size 2180, which is longer than the specified 1500
Created a chunk of size 1692, which is longer than the specified 1500
Created a chunk of size 1597, which is longer than the specified 1500
Created a chunk of size 1776, which is longer than the specified 1500
Created a chunk of size 1681, which is longer than the specified 1500
Created a chunk of size 1732, which is longer than the specified 1500
Created a chunk of size 1866, which is longer than the specified 1500
Created a chunk of s

In [50]:
documents[0]

Document(metadata={'source': 'tagged_desc.txt'}, page_content='BK000001 Harry Potter thinks he is an ordinary boy - until he is rescued by an owl, taken to Hogwarts School of Witchcraft and Wizardry, learns to play Quidditch and does battle in a deadly duel. The Reason ... HARRY POTTER IS A WIZARD!\nBK000002 24 deltagare. Bara en överlever. Varje år direktsänds Hungerspelen som en av de största mediala händelserna. Från varje distrikt väljs två ungdomar mellan tolv och arton ut att delta. De lottas fram, och skickas omedelbart till huvudstaden där de ska stajlas, tränas och visas upp i direktsända intervjuer. De måste se till att vinna folkets gunst för att få sponsorer. Varje år konstrueras en ny spelplats där ungdomarna släpps ut. Men bara den som överlever får åka hem! Katniss Everdeen är fjorton år. Hon älskar sin lillasyster Prim över allt annat, och försöker skydda henne från allt hemskt i distrikt tolv, men mot Hungerspelen har hon inget skydd. Det namn som dras i lottningen i d

In [51]:
import os
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs = {'device':'cuda'}
)
db_books = FAISS.from_documents(
    documents=documents,
    embedding=embeddings
)

In [56]:
query = "A book to know about Harry Potter"
docs = db_books.similarity_search(query,k=5)

for doc in docs:
    print("\nDocuments ID:",doc.id)
    print("Metadata:",doc.metadata)
    print("Content:",doc.page_content[:100])
    print("-"*50)
    


Documents ID: 233988df-0f79-401a-ac4b-37d6d597fa4b
Metadata: {'source': 'tagged_desc.txt'}
Content: "BK000012 Since its immediate success in 1813, Pride and Prejudice has remained one of the most popu
--------------------------------------------------

Documents ID: 2d6af209-773d-489d-abd1-0e74bd7bea93
Metadata: {'source': 'tagged_desc.txt'}
Content: BK000459 Alternate cover edition: 9780439709101From internationally acclaimed storyteller Cornelia F
--------------------------------------------------

Documents ID: 3848fedf-e9b2-4f25-9991-843b83b5168c
Metadata: {'source': 'tagged_desc.txt'}
Content: BK001210 J.K. Rowling's screenwriting debut is captured in this exciting hardcover edition of the Fa
--------------------------------------------------

Documents ID: db2b94e9-326b-45e6-bb67-6a784d1ac2b2
Metadata: {'source': 'tagged_desc.txt'}
Content: BK000298 Librarian note: An alternate cover for this edition can be found here: 2005.Journeys to the
---------------------------------------

In [57]:
def similar_recommendations(query: str, top_k: int = 10) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k=5)
    books_list = []
    
    for rec in recs:
        first_word = rec.page_content.strip().split()[0]  
        
        if first_word.startswith("BK") and first_word[2:].isdigit():
            books_list.append(first_word)  
        else:
            print(f"Skipping document with invalid or missing BookID: {rec.id}")
    
    return df[df["FormattedBookID"].isin(books_list)].head(top_k)



In [58]:
similar_recommendations("Harry Potter")

Skipping document with invalid or missing BookID: 233988df-0f79-401a-ac4b-37d6d597fa4b


,BookID,FormattedBookID,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL,tagged_desc
0,1,BK000001,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,9278135,https://www.goodreads.com/book/show/72193.Harr...,BK000001 Harry Potter thinks he is an ordinary...
18,19,BK000019,Harry Potter and the Deathly Hallows (Harry Po...,J.K. Rowling,"Harry has been burdened with a dark, dangerous...","['Fantasy', 'Young Adult', 'Fiction', 'Magic',...",4.62,3468276,https://www.goodreads.com/book/show/136251.Har...,"BK000019 Harry has been burdened with a dark, ..."
22,23,BK000023,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,Harry Potter is about to start his fifth year ...,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...",4.50,3147846,https://www.goodreads.com/book/show/2.Harry_Po...,BK000023 Harry Potter is about to start his fi...
1209,1210,BK001210,Fantastic Beasts and Where to Find Them: The O...,J.K. Rowling,J.K. Rowling's screenwriting debut is captured...,"['Fantasy', 'Fiction', 'Young Adult', 'Plays',...",4.18,153274,https://www.goodreads.com/book/show/29363501-f...,BK001210 J.K. Rowling's screenwriting debut is...
